# 🧠 DSPy Checklist: Minimalist Demonstration

This notebook demonstrates the core building blocks of DSPy: **Signatures** and **Modules** (like Chain of Thought).

In [1]:
import sys
import os
# Add the project root to sys.path
# '..' assumes the notebook is in notebooks/ and src/ is in the parent dir
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [3]:
import dspy
from dspy import settings, Evaluate
from dotenv import load_dotenv
from src.retriever import HDBRetriever, get_hdb_index
from src.model import HDBRAG

# 1. Setup - Ensure you have OPENAI_API_KEY in your .env file
load_dotenv()

student = dspy.LM('ollama/qwen3:0.6b', api_base='http://localhost:11434')
judge_lm = dspy.LM('ollama/qwen3:0.6b', api_base='http://localhost:11434', cache=True, max_tokens=512, temperature=0)

settings.configure(lm=student)
student

In [4]:
import json

with open("../data/qa_pairs.json", "r", encoding="utf-8") as f:
    qa_pairs = json.load(f)

print(f"Loaded {len(qa_pairs)} QA pairs")
qa_pairs[0]

examples = [
    dspy.Example(
        question=qa["question"],
        answer=qa["answer"]
    ).with_inputs("question")
    for qa in qa_pairs
]
print(examples[1])
print("Inputs:", examples[0].inputs())
print("Label:", examples[0].answer)
train_examples = examples[:30]
test_examples = examples[50:]

Loaded 130 QA pairs
Example({'question': 'If I take two housing subsidies, am I eligible for CPF housing grants?', 'answer': 'Taking two housing subsidies means eligibility depends on the number, and second-timers need to pay their respective resale levies when buying a flat from HDB. Seniors may apply for a short-lease flat, and if someone owns a flat, they must dispose of it.'}) (input_keys={'question'})
Inputs: Example({'question': 'What are the eligibility criteria for HDB grants for Singapore citizens who are unmarried, widowed, or divorced, aged 35 and above, and buying a flat on their own?'}) (input_keys={'question'})
Label: The eligibility criteria for HDB grants include being a Singapore citizen, unmarried, widowed, divorced, aged 35 and above, and purchasing a flat on your own. You can apply via the HDB Flat Portal for a holistic understanding of your eligibility. For specific eligibility details, refer to the section on seniors if you're planning a 35+ or senior-friendly fla

In [5]:
class JudgeQA(dspy.Signature):
    """
    Decide whether the predicted answer is correct.

    Mark as correct if:
    - It states the same fact as the gold answer, OR
    - Both answers indicate that the information is missing, unknown,
      not mentioned, or cannot be determined from the context.

    Mark as incorrect if:
    - The prediction contradicts the gold answer
    - The prediction invents information
    """
    question: str = dspy.InputField()
    gold_answer: str = dspy.InputField()
    predicted_answer: str = dspy.InputField()

    is_accurate: bool = dspy.OutputField(
        desc="True if the predicted answer is semantically equivalent to the gold answer"
    )

judge = dspy.Predict(JudgeQA)
def metric(example, pred, trace=None):
    # Guard against bad predictions
    if not hasattr(pred, "answer"):
        return 0.0
    if pred.answer is None or pred.answer.strip() == "":
        return 0.0

    try:
        with dspy.settings.context(lm=judge_lm):
            result = judge(
                question=example.question,
                gold_answer=example.answer,
                predicted_answer=pred.answer
            )
        # Coerce to float for Evaluate
        return float(result.is_accurate)

    except Exception as e:
        # Any judge failure must NOT crash Evaluate
        if trace is not None:
            trace["judge_error"] = str(e)
        return 0.0

In [6]:
# 2. Initialize the index and the simple RAG module
index = get_hdb_index()
rag = HDBRAG(index=index, k=3)

trace = {}
# 3. Perform a sample query
query = examples[1].question
print(f"\nQuery: {query}")

response = rag(question=query)
print("\n--- Answer ---")
print(response.answer)
print(type(response.answer))

Loading weights:   0%|          | 0/41 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-TinyBERT-L-2-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Query: If I take two housing subsidies, am I eligible for CPF housing grants?

--- Answer ---
Taking two housing subsidies may qualify you as a second-timer, but eligibility for CPF housing grants is determined by other factors, such as the number of core members.
<class 'str'>


In [7]:
evaluator = dspy.Evaluate(
    devset=test_examples,
    metric=metric,
    num_threads=4,
    display_progress=True,
    display_table=True
)

results = evaluator(rag)

Average Metric: 2.00 / 4 (50.0%):   5%|▌         | 4/80 [00:44<09:11,  7.25s/it]

2026/02/19 23:38:16 ERROR dspy.utils.parallelizer: Error for Example({'question': None, 'answer': None}) (input_keys={'question'}): 2 validation errors for RetrievalStartEvent
str_or_query_bundle.str
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
str_or_query_bundle.QueryBundle
  Input should be a dictionary or an instance of QueryBundle [type=dataclass_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/dataclass_type. Set `provide_traceback=True` for traceback.


Average Metric: 7.00 / 14 (50.0%):  19%|█▉        | 15/80 [03:06<12:44, 11.77s/it]

2026/02/19 23:40:19 ERROR dspy.utils.parallelizer: Error for Example({'question': None, 'answer': None}) (input_keys={'question'}): 2 validation errors for RetrievalStartEvent
str_or_query_bundle.str
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
str_or_query_bundle.QueryBundle
  Input should be a dictionary or an instance of QueryBundle [type=dataclass_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/dataclass_type. Set `provide_traceback=True` for traceback.


Average Metric: 42.00 / 78 (53.8%): 100%|██████████| 80/80 [19:21<00:00, 14.52s/it]

2026/02/19 23:56:35 INFO dspy.evaluate.evaluate: Average Metric: 42.0 / 80 (52.5%)


,question,example_answer,context,pred_answer,metric,answer
0,How does the EASE program enhance accessibility in HDB flats for a...,"The EASE program enhances mobility and comfort in HDB flats, while...",[Prediction( long_text='[For Our Seniors] Get an overview of the E...,The EASE program enhances accessibility in HDB flats for active se...,✔️ [1.000],NaN
1,"""Can I apply for a 3Gen flat as a core family nucleus, and what ar...","""To apply for a 3Gen flat, you must be a Singapore Citizen (SC) an...",[Prediction( long_text='[Core family nucleus] A household’s eligib...,"Yes, you can apply for a 3Gen flat as a core family nucleus. The e...",✔️ [0.000],NaN
2,A user is seeking information about eligibility for HDB BTO grants.,Eligible flat buyers must be 55+ at the time of their HFE letter a...,[Prediction( long_text='[Eligibility conditions] ... ccount any HD...,The user is eligible for CPF housing grants as outlined in the con...,✔️ [0.000],NaN
3,What steps should sellers take before the resale application is ap...,Sellers must submit the resale application after buyers have exerc...,[Prediction( long_text='[Acceptance and Approval] Find out what ac...,Sellers should take the following steps: 1. Complete the acceptanc...,✔️ [1.000],NaN
4,Can I apply for HDB housing grants if my core family includes my f...,"Yes, your core family nucleus, which includes you and your fiancé,...",[Prediction( long_text='[Core family nucleus] A household’s eligib...,"Yes, you can apply for HDB housing grants if your core family incl...",✔️ [1.000],NaN
...,...,...,...,...,...,...
75,What are the grant conditions for HDB BTO?,The grant conditions apply to properties held on trust with the us...,[Prediction( long_text='[Eligibility conditions] ... us\xa0or Prim...,The grant conditions for HDB BTO are: - Applicants must meet the i...,✔️ [0.000],NaN
76,Can seniors apply for the Proximity Housing Grant for their HDB BT...,Seniors aged 65 and above or individuals with an interest in a 4-r...,[Prediction( long_text='[Seniors] Understand the eligibility condi...,"Yes, seniors can apply for the Proximity Housing Grant for their H...",✔️ [1.000],NaN
77,"""Can I apply for an HFE letter online and receive notifications on...","""Based on the context, you can apply for an HFE letter online for ...",[Prediction( long_text='[When to apply] Plan and apply for the HFE...,"Yes, you can apply for an HFE letter online and receive notificati...",✔️ [1.000],NaN
78,A user asking about eligibility for HDB BTO grants.,"To apply for an HDB BTO grant, applicants must not own or have an ...",[Prediction( long_text='[Modes of Sale] Our modes of sale are: Bui...,The user is eligible for HDB BTO grants. They can apply directly f...,✔️ [0.000],NaN


In [8]:
results

EvaluationResult(score=50.0, results=<list of 50 results>)

## Optimization

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

rag_save_path = "../data/optimized_rag_qwen0-6b.json"

teleprompter = dspy.MIPROv2(
    metric=metric,
    max_bootstrapped_demos=4,
    max_labeled_demos=5,
    num_threads=4
)

optimized_rag = teleprompter.compile(
    rag,
    trainset=train_examples,
)

optimized_rag.save(rag_save_path)

evaluator = dspy.Evaluate(
    devset=test_examples,
    metric=metric,
    num_threads=4,
)
results = evaluator(optimized_rag)
results

In [ ]:
evaluator = dspy.Evaluate(
    devset=test_examples,
    metric=metric,
    num_threads=4,
    # display_progress=True,
    # display_table=True
)

results = evaluator(rag)

optimized_rag = HDBRAG(index=index, k=3)

optimized_rag.load("../data/optimized_rag_qwen3:0.6b.json")

evaluator = dspy.Evaluate(
    devset=test_examples,
    metric=metric,
    num_threads=4,
    # display_progress=True,
)

results = evaluator(optimized_rag)